In [11]:
import flwr as fl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import os
import itertools

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(False)
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Num GPUs Available:  0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [9]:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [10]:
#model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
#model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

In [6]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

model = Sequential([
    Input(shape=(2)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])

test_Y = np.array([
    [1],
    [1],
    [0],
    [0]
])



Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 15        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


Epoch 1/4
1000/1000 [==============================] - 6s 5ms/step - loss: 0.6081 - accuracy: 0.7456
Epoch 2/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.4088 - accuracy: 0.9298
Epoch 3/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2770 - accuracy: 0.9727
Epoch 4/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.2025 - accuracy: 0.9880
[[0.71]
 [0.72]
 [0.10]
 [0.10]]


In [10]:
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]

In [12]:
class CifarClient(fl.client.NumPyClient):
    def get_parameters(self, config):
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        #model.fit(x_train, y_train, epochs=1, batch_size=32, steps_per_epoch=3)

        print("executando fit")
        model.fit(
            train_dataset,
            steps_per_epoch=1000,
            epochs=4
        )

        return model.get_weights(), 2, {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        
        print("executando set")
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset_gen
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        tflite_model = converter.convert()
        with open('model.tflite', 'wb') as f:
            f.write(tflite_model)
        
        loss, accuracy = model.evaluate(test_X, test_Y)
        return loss, len(test_X), {"accuracy": float(accuracy)}

In [13]:
fl.client.start_numpy_client(server_address="localhost:8080", client=CifarClient())

INFO flwr 2023-07-05 17:02:53,079 | grpc.py:50 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-07-05 17:02:53,110 | connection.py:39 | ChannelConnectivity.IDLE
DEBUG flwr 2023-07-05 17:02:53,113 | connection.py:39 | ChannelConnectivity.READY


executando fit
Epoch 1/4
1000/1000 [==============================] - 6s 5ms/step - loss: 0.0430 - accuracy: 0.9985
Epoch 2/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0412 - accuracy: 0.9982
Epoch 3/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0396 - accuracy: 0.9984
Epoch 4/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0377 - accuracy: 0.9984
executando set


INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp5oi1whry\assets


INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmp5oi1whry\assets
c:\Program Files\Conda\envs\ubq\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


1/1 [==============================] - 0s 33ms/step - loss: 0.0082 - accuracy: 1.0000
executando fit
Epoch 1/4
1000/1000 [==============================] - 6s 6ms/step - loss: 0.0379 - accuracy: 0.9980
Epoch 2/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0362 - accuracy: 0.9982
Epoch 3/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0346 - accuracy: 0.9986
Epoch 4/4
1000/1000 [==============================] - 5s 5ms/step - loss: 0.0342 - accuracy: 0.9985
executando set


...

INFO:tensorflow:Assets written to: C:\Users\custo\AppData\Local\Temp\tmpnvgkxntt\assets


1/1 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 1.0000

c:\Program Files\Conda\envs\ubq\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


1/1 [==============================] - 0s 24ms/step - loss: 0.0029 - accuracy: 1.0000


DEBUG flwr 2023-07-05 17:05:00,376 | connection.py:113 | gRPC channel closed
DEBUG:flwr:gRPC channel closed
INFO flwr 2023-07-05 17:05:00,378 | app.py:185 | Disconnect and shut down
INFO:flwr:Disconnect and shut down
